In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

ANN Prototype

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
    layers.Input(shape = (128,)),
    layers.Normalization(mean = 127, variance = 77.4),
    layers.Dense(100, activation='tanh'), #86
    layers.Dense(80, activation='tanh'), #64
    layers.Dense(60, activation='tanh'), #42
    layers.Dense(40, activation='tanh'), #21
    layers.Dense(20, activation='relu'),
    layers.Dense(1, activation='relu')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 2

Test training

In [9]:
save_to_disk = True # Set to FALSE if we don't want this to overwrite the current model.

TrainFeatures = pd.read_csv('../data/trainFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets = pd.read_csv('../data/trainTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
TrainTargets.iloc[:,0] = TrainTargets.iloc[:,0].astype(float)

EvalFeatures = pd.read_csv('../data/evalFeaturesNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets = pd.read_csv('../data/evalTargetsNewSplit.csv', header = 0, index_col = 0).drop(['video_id','time_stamp'], axis = 'columns')
EvalTargets.iloc[:,0] = EvalTargets.iloc[:,0].astype(float)

# Since our dataset is imbalanced, we need a weighted binary cross entropy loss function.
def weighted_bce(y_true, y_pred):
    weights = (y_true * 4) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = tf.math.reduce_mean(bce * weights)
    return weighted_bce

# Here, we choose the loss function to minimize, the optimizer algorithm, and additional loss metrics to display on screen.
model.compile(loss = weighted_bce, optimizer = 'adam', metrics = ['accuracy'])

# Time to train! Epochs = # times the training algorithm goes through the whole set.
# Batch size = # elements the algorithm "sees" at a time between adjusting model parameters.
model.fit(TrainFeatures,TrainTargets,epochs = 12, batch_size = 8)

if save_to_disk:
    # Saving model to JSON and weights to H5.
    model_json = model.to_json()
    with open("../models/model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("../models/model.h5")
    print("Saved model to disk")

# Measure accuracy on the evaluation set.
loss  = model.evaluate(EvalFeatures, EvalTargets)
print('Loss on Validation Set: ', loss)
print('Confusion matrix:')
predictions = np.round(model.predict(EvalFeatures))
true_labels = EvalTargets.iloc[:,0]

m = tf.keras.metrics.Recall()
m.update_state(true_labels, predictions)
print('Recall: ' + str(m.result().numpy()))

conf_matrix = tf.math.confusion_matrix(true_labels, predictions)
conf_matrix

Epoch 1/12
7993/7993 [==============================] - 9s 1ms/step - loss: 0.7559 - accuracy: 0.8522
Epoch 2/12
7993/7993 [==============================] - 8s 1ms/step - loss: 0.7013 - accuracy: 0.8547
Epoch 3/12
7993/7993 [==============================] - 9s 1ms/step - loss: 0.7558 - accuracy: 0.8537
Epoch 4/12
7993/7993 [==============================] - 7s 922us/step - loss: 0.6775 - accuracy: 0.8582
Epoch 5/12
7993/7993 [==============================] - 8s 999us/step - loss: 0.6751 - accuracy: 0.8542
Epoch 6/12
7993/7993 [==============================] - 9s 1ms/step - loss: 0.6510 - accuracy: 0.8567
Epoch 7/12
7993/7993 [==============================] - 7s 930us/step - loss: 0.6514 - accuracy: 0.8546
Epoch 8/12
7993/7993 [==============================] - 8s 944us/step - loss: 0.7320 - accuracy: 0.8545
Epoch 9/12
7993/7993 [==============================] - 8s 942us/step - loss: 0.7193 - accuracy: 0.8561
Epoch 10/12
7993/7993 [==============================] - 8s 969us/step -

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[9280,  945],
       [1881,  683]])>